In [6]:
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.oneplane import OnePlaneEncoder

from dlgo.networks import small
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Layer, ZeroPadding2D, Conv2D, Softmax, Activation
from tensorflow.keras.layers import Concatenate, Conv2DTranspose, BatchNormalization, add
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.experimental.numpy import moveaxis


from tensorflow.keras.layers import Flatten
from p4conv_slow import P4Conv2D, P4MaxPooling2D



In [2]:
"""
Change Channels First to channels Last
"""
class Flip(Layer):
    def __init__(self):
        super(Flip,self).__init__(name='flip')
    def call(self,input_tensor):
        return moveaxis(input_tensor, [1,2,3],[3,1,2])


In [3]:
go_board_rows, go_board_cols = 19, 19
num_classes = go_board_rows * go_board_cols
num_games = 100

encoder = OnePlaneEncoder((go_board_rows, go_board_cols))

#to loop through zip files and create features/labels for training
processor = GoDataProcessor(encoder=encoder.name())

#create generator to loop through training/testing data
generator = processor.load_go_data('train', num_games, use_generator=True)
test_generator = processor.load_go_data('test',num_games, use_generator= True)
print(generator.get_num_samples(),'moves in one epoch')
print(generator.get_num_samples()/128,'steps per epoch')


>>> Reading cached index page
KGS-2019_04-19-1255-.tar.gz 1255
KGS-2019_03-19-1478-.tar.gz 1478
KGS-2019_02-19-1412-.tar.gz 1412
KGS-2019_01-19-2095-.tar.gz 2095
KGS-2018_12-19-1992-.tar.gz 1992
KGS-2018_11-19-1879-.tar.gz 1879
KGS-2018_10-19-1209-.tar.gz 1209
KGS-2018_09-19-1587-.tar.gz 1587
KGS-2018_08-19-1447-.tar.gz 1447
KGS-2018_07-19-949-.tar.gz 949
KGS-2018_06-19-1002-.tar.gz 1002
KGS-2018_05-19-1590-.tar.gz 1590
KGS-2018_04-19-1612-.tar.gz 1612
KGS-2018_03-19-833-.tar.gz 833
KGS-2018_02-19-1167-.tar.gz 1167
KGS-2018_01-19-1526-.tar.gz 1526
KGS-2017_12-19-1488-.tar.gz 1488
KGS-2017_11-19-945-.tar.gz 945
KGS-2017_10-19-1351-.tar.gz 1351
KGS-2017_09-19-1353-.tar.gz 1353
KGS-2017_08-19-2205-.tar.gz 2205
KGS-2017_07-19-1191-.tar.gz 1191
KGS-2017_06-19-910-.tar.gz 910
KGS-2017_05-19-847-.tar.gz 847
KGS-2017_04-19-913-.tar.gz 913
KGS-2017_03-19-717-.tar.gz 717
KGS-2017_02-19-525-.tar.gz 525
KGS-2017_01-19-733-.tar.gz 733
KGS-2016_12-19-1208-.tar.gz 1208
KGS-2016_11-19-980-.tar.gz 980


## Fully-Convolutional VGG-type CNN

In [19]:
#define keras model
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Flip())

#three 3x3 convs has a receptive field equivalent to one 7x7
model.add(Conv2D(filters = 16, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(Conv2D(filters = 16, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(Conv2D(filters = 32, kernel_size = (3, 3),activation='relu',padding = 'same'))


#two 3x3 convs has a receptive field equivalent to one 5x5
model.add(Conv2D(filters = 32, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(Conv2D(filters = 50, kernel_size = (3, 3),activation='relu',padding = 'same'))


model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu',padding='same'))

#obtain prediction
model.add(Conv2D(filters = 1, kernel_size = (1,1), activation = 'relu'))

model.add(Flatten())
model.add(Softmax())

#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=CategoricalCrossentropy(), optimizer='adam',metrics=['accuracy'])

model.build((None,*input_shape))

#print model summary
print(model.summary())


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flip (Flip)                 (None, 19, 19, 1)         0         
                                                                 
 conv2d_50 (Conv2D)          (None, 19, 19, 16)        160       
                                                                 
 conv2d_51 (Conv2D)          (None, 19, 19, 16)        2320      
                                                                 
 conv2d_52 (Conv2D)          (None, 19, 19, 32)        4640      
                                                                 
 conv2d_53 (Conv2D)          (None, 19, 19, 32)        9248      
                                                                 
 conv2d_54 (Conv2D)          (None, 19, 19, 50)        14450     
                                                                 
 conv2d_55 (Conv2D)          (None, 19, 19, 128)     

### Train model

In [20]:
epochs = 50
batch_size = 128
model.fit(
    generator.generate(batch_size, num_classes),
    steps_per_epoch = generator.get_num_samples()//batch_size,
    epochs = epochs,
    validation_data = test_generator.generate(batch_size, num_classes),
    validation_steps = test_generator.get_num_samples()//batch_size,
    #callbacks=[
    #ModelCheckpoint('checkpoints/small_model_epoch_{epoch}.h5')
    #] #callback stores model at each epoch
    )


Epoch 1/50
329/329 [==============================] - 220s 666ms/step - loss: 4.7108 - accuracy: 0.0838 - val_loss: 4.0611 - val_accuracy: 0.1404
Epoch 2/50
329/329 [==============================] - 211s 642ms/step - loss: 3.9289 - accuracy: 0.1517 - val_loss: 3.8119 - val_accuracy: 0.1671
Epoch 3/50
329/329 [==============================] - 210s 638ms/step - loss: 3.7270 - accuracy: 0.1744 - val_loss: 3.6698 - val_accuracy: 0.1821
Epoch 4/50
329/329 [==============================] - 211s 640ms/step - loss: 3.5798 - accuracy: 0.1948 - val_loss: 3.5433 - val_accuracy: 0.1985
Epoch 5/50
329/329 [==============================] - 204s 621ms/step - loss: 3.4693 - accuracy: 0.2099 - val_loss: 3.4337 - val_accuracy: 0.2125
Epoch 6/50
329/329 [==============================] - 211s 643ms/step - loss: 3.3737 - accuracy: 0.2247 - val_loss: 3.3883 - val_accuracy: 0.2195
Epoch 7/50
329/329 [==============================] - 211s 643ms/step - loss: 3.2909 - accuracy: 0.2382 - val_loss: 3.3484 -

KeyboardInterrupt: 

## Build ResNet block

In [7]:
def residual_block(x, filters, kernel_size):
    #Residual path
    conv1 = Conv2D(filters = filters, kernel_size = kernel_size, padding = 'same')(x)
    bn1 = BatchNormalization(axis = -1)(conv1)
    relu1 = Activation('relu')(bn1)
    conv2 = Conv2D(filters = filters, kernel_size = kernel_size, padding = 'same')(relu1)
    residual = BatchNormalization(axis = -1)(conv2)
    
    #add residual to skip connection
    return Activation('relu')(add([x,residual]))

## Use Resnet block to enhance previous VGG-type model

In [12]:
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)

inputs = Input(shape = input_shape) #input is 19x19
res0 = Flip()(inputs)

res1 = residual_block(res0, filters = 16, kernel_size = (3,3))
res2 = residual_block(res1, filters = 16, kernel_size = (3,3))

expand1 = Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same')(res2)
bottleneck1 = Conv2D(filters = 32, kernel_size = (1,1), activation = 'relu')(res2)

res3 = residual_block(bottleneck1, filters = 32, kernel_size = (3,3))
res4 = residual_block(res3, filters = 32, kernel_size = (3,3))

expand1 = Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same')(res4)
bottleneck2 = Conv2D(filters = 64, kernel_size = (1,1), activation = 'relu')(res4)


res5 = residual_block(bottleneck2, filters = 64, kernel_size = (3,3))
res6 = residual_block(res5, filters = 64, kernel_size = (3,3))



#obtain prediction
pred1 = Conv2D(filters = 1, kernel_size = (1,1), activation = 'relu')(res6)

prediction = Softmax()(Flatten()(pred1))

model = Model(inputs = inputs, outputs = prediction)

model.compile(loss=CategoricalCrossentropy(), optimizer='adam',metrics=['accuracy'])

model.build((None,*input_shape))

#print model summary
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 1, 19, 19)]  0           []                               
                                                                                                  
 flip (Flip)                    (None, 19, 19, 1)    0           ['input_5[0][0]']                
                                                                                                  
 conv2d_39 (Conv2D)             (None, 19, 19, 16)   160         ['flip[0][0]']                   
                                                                                                  
 batch_normalization_34 (BatchN  (None, 19, 19, 16)  64          ['conv2d_39[0][0]']              
 ormalization)                                                                              

                                                                                                  
 activation_42 (Activation)     (None, 19, 19, 64)   0           ['batch_normalization_42[0][0]'] 
                                                                                                  
 conv2d_52 (Conv2D)             (None, 19, 19, 64)   36928       ['activation_42[0][0]']          
                                                                                                  
 batch_normalization_43 (BatchN  (None, 19, 19, 64)  256         ['conv2d_52[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_21 (Add)                   (None, 19, 19, 64)   0           ['conv2d_50[0][0]',              
                                                                  'batch_normalization_43[0][0]'] 
          

### Train deeper Resnet-type model

In [13]:
epochs = 50
batch_size = 128
model.fit(
    generator.generate(batch_size, num_classes),
    steps_per_epoch = generator.get_num_samples()//batch_size,
    epochs = epochs,
    validation_data = test_generator.generate(batch_size, num_classes),
    validation_steps = test_generator.get_num_samples()//batch_size,
    #callbacks=[
    #ModelCheckpoint('checkpoints/small_model_epoch_{epoch}.h5')
    #] #callback stores model at each epoch
    )


Epoch 1/50
329/329 [==============================] - 437s 1s/step - loss: 4.9447 - accuracy: 0.0737 - val_loss: 4.7674 - val_accuracy: 0.0915
Epoch 2/50
329/329 [==============================] - 512s 2s/step - loss: 3.7270 - accuracy: 0.1729 - val_loss: 3.6492 - val_accuracy: 0.1795
Epoch 3/50
329/329 [==============================] - 401s 1s/step - loss: 3.3917 - accuracy: 0.2216 - val_loss: 3.4601 - val_accuracy: 0.2089
Epoch 4/50
329/329 [==============================] - 390s 1s/step - loss: 3.1864 - accuracy: 0.2561 - val_loss: 3.2672 - val_accuracy: 0.2347
Epoch 5/50
329/329 [==============================] - 388s 1s/step - loss: 3.0127 - accuracy: 0.2865 - val_loss: 3.1283 - val_accuracy: 0.2577
Epoch 6/50
329/329 [==============================] - 389s 1s/step - loss: 2.8576 - accuracy: 0.3164 - val_loss: 3.0679 - val_accuracy: 0.2746
Epoch 7/50
329/329 [==============================] - 387s 1s/step - loss: 2.7068 - accuracy: 0.3502 - val_loss: 3.1089 - val_accuracy: 0.2748

KeyboardInterrupt: 

## Build U-Net-esque network

In [69]:
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)

inputs = Input(shape = input_shape) #input is 19x19
conv1 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(Flip()(inputs)) 
conv1a = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(conv1)

conv1b = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu')(conv1a) #output is 17x17

conv2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu')(conv1b) #output is 15x15

conv3 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu')(conv2) #output is 13x13

conv4 = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu')(conv3) #output is 11x11 (Correct size)
conv5 = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu')(conv4) #output is 9x9

conv6 = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu')(conv5) #output is 7x7

#rescale 7x7 to 11x11
rescale9x9_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(conv6) #output is 9x9
rescale9x9_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale9x9_1)

rescale11x11_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(rescale9x9_2) #output is 11x11
rescale11x11_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale11x11_1)

#Concat 11x11's together
concat11x11_1 = Concatenate()([rescale11x11_2,conv4])
concat11x11_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(concat11x11_1)

#rescale 11x11 to 15x15
rescale13x13_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(concat11x11_2) #output is 13x13
rescale13x13_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale13x13_1)

rescale15x15_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(rescale13x13_2) #output is 15x15
rescale15x15_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale15x15_1)

#Concat 15x15's together
concat15x15_1 = Concatenate()([rescale15x15_2,conv2])
concat15x15_2 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(concat15x15_1)

#rescale 15x15 to 19x19
rescale17x17_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(concat15x15_2) #output is 17x17
rescale17x17_2 = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale17x17_1)

rescale19x19_1 = Conv2DTranspose(filters = 16, kernel_size = (3,3), activation = 'relu')(rescale17x17_2) #output is 19x19
rescale19x19_2 = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same')(rescale19x19_1)

#Concat 19x19's together
concat19x19_1 = Concatenate()([rescale19x19_2, conv1a])

#Add extra convolutional layers
final_1 = Conv2D(filters = 16, kernel_size = (3,3), activation = 'relu', padding = 'same')(concat19x19_1)
final_2 = Conv2D(filters = 16, kernel_size = (1,1), activation = 'relu')(final_1)
final_3 = Conv2D(filters = 1, kernel_size = (1,1), activation = 'relu')(final_2)

predictions = Softmax()(Flatten()(final_3))

model = Model(inputs = inputs, outputs = predictions)

print(model.summary())

model.compile(loss=CategoricalCrossentropy(), optimizer='adam',metrics=['accuracy'])

model.build((None,*input_shape))


Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, 1, 19, 19)]  0           []                               
                                                                                                  
 flip (Flip)                    (None, 19, 19, 1)    0           ['input_54[0][0]']               
                                                                                                  
 conv2d_467 (Conv2D)            (None, 19, 19, 16)   160         ['flip[0][0]']                   
                                                                                                  
 conv2d_468 (Conv2D)            (None, 19, 19, 16)   2320        ['conv2d_467[0][0]']             
                                                                                           

### Train U-net type model

In [70]:
epochs = 50
batch_size = 128
model.fit(
    generator.generate(batch_size, num_classes),
    steps_per_epoch = generator.get_num_samples()//batch_size,
    epochs = epochs,
    validation_data = test_generator.generate(batch_size, num_classes),
    validation_steps = test_generator.get_num_samples()//batch_size,
    #callbacks=[
    #ModelCheckpoint('checkpoints/small_model_epoch_{epoch}.h5')
    #] #callback stores model at each epoch
    )


Epoch 1/50
329/329 [==============================] - 186s 560ms/step - loss: 4.9081 - accuracy: 0.0634 - val_loss: 4.0826 - val_accuracy: 0.1214
Epoch 2/50
329/329 [==============================] - 169s 516ms/step - loss: 3.9704 - accuracy: 0.1353 - val_loss: 3.8611 - val_accuracy: 0.1492
Epoch 3/50
329/329 [==============================] - 168s 511ms/step - loss: 3.8095 - accuracy: 0.1556 - val_loss: 3.7653 - val_accuracy: 0.1588
Epoch 4/50
329/329 [==============================] - 149s 453ms/step - loss: 3.7058 - accuracy: 0.1685 - val_loss: 3.6604 - val_accuracy: 0.1706
Epoch 5/50
329/329 [==============================] - 149s 454ms/step - loss: 3.6228 - accuracy: 0.1789 - val_loss: 3.6001 - val_accuracy: 0.1779
Epoch 6/50
329/329 [==============================] - 148s 451ms/step - loss: 3.5521 - accuracy: 0.1880 - val_loss: 3.5820 - val_accuracy: 0.1802
Epoch 7/50
329/329 [==============================] - 152s 461ms/step - loss: 3.4905 - accuracy: 0.1951 - val_loss: 3.5429 -

KeyboardInterrupt: 

## P4-equivariant VGG-type CNN

In [9]:
#define keras model
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Flip())

#small_layers = small.layers(input_shape)
#for layer in small_layers:
#    model.add(layer)

#group convolution

#three 3x3 convs has a receptive field equivalent to one 7x7
model.add(P4Conv2D(num_channels = 8, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(P4Conv2D(num_channels = 8, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(P4Conv2D(num_channels = 8, kernel_size = (3, 3),activation='relu',padding = 'same'))

#model.add(P4Conv2D(num_channels = 6, kernel_size = (1, 1),activation='relu'))


#two 3x3 convs has a receptive field equivalent to one 5x5
model.add(P4Conv2D(num_channels = 16, kernel_size = (3, 3),activation='relu',padding = 'same'))
model.add(P4Conv2D(num_channels = 16, kernel_size = (3, 3),activation='relu',padding = 'same'))

#bottleneck to reduce parameters
#model.add(P4Conv2D(num_channels = 8, kernel_size = (1, 1),activation='relu'))


#two 3x3 convs has a receptive field equivalent to one 5x5
#model.add(P4Conv2D(num_channels = 32, kernel_size = (3, 3),activation='relu',padding = 'same'))
#model.add(P4Conv2D(num_channels = 32, kernel_size = (3, 3),activation='relu',padding = 'same'))

#bottleneck to reduce parameters
#model.add(P4Conv2D(num_channels = 8, kernel_size = (1, 1),activation='relu'))


#two 3x3 convs has a receptive field equivalent to one 5x5
#model.add(P4Conv2D(num_channels = 64, kernel_size = (3, 3),activation='relu',padding = 'same'))
#model.add(P4Conv2D(num_channels = 64, kernel_size = (3, 3),activation='relu',padding = 'same'))

#bottleneck to reduce parameters
#model.add(P4Conv2D(num_channels = 8, kernel_size = (1, 1),activation='relu'))

#switch feature volume from being a function of the space group to the translation subgroup
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu',padding='same'))

#obtain prediction
model.add(Conv2D(filters = 1, kernel_size = (1,1), activation = 'relu'))

model.add(Flatten())
model.add(Softmax())

#model.add(Dense(100, activation='relu'))
#model.add(Dense(num_classes, activation='softmax'))



model.compile(loss=CategoricalCrossentropy(), optimizer='adam',metrics=['accuracy'])

model.build((None,*input_shape))

#print model summary
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flip (Flip)                 (None, 19, 19, 1)         0         
                                                                 
 p4_conv2d_27 (P4Conv2D)     (None, 19, 19, 32)        72        
                                                                 
 p4_conv2d_28 (P4Conv2D)     (None, 19, 19, 32)        2304      
                                                                 
 p4_conv2d_29 (P4Conv2D)     (None, 19, 19, 32)        2304      
                                                                 
 p4_conv2d_30 (P4Conv2D)     (None, 19, 19, 64)        4608      
                                                                 
 p4_conv2d_31 (P4Conv2D)     (None, 19, 19, 64)        9216      
                                                                 
 conv2d_6 (Conv2D)           (None, 19, 19, 128)      

### Train model

In [10]:
epochs = 50
batch_size = 128
model.fit(
    generator.generate(batch_size, num_classes),
    steps_per_epoch = generator.get_num_samples()//batch_size,
    epochs = epochs,
    validation_data = test_generator.generate(batch_size, num_classes),
    validation_steps = test_generator.get_num_samples()//batch_size,
    #callbacks=[
    #ModelCheckpoint('checkpoints/small_model_epoch_{epoch}.h5')
    #] #callback stores model at each epoch
    )


Epoch 1/50


2023-01-21 15:11:02.096660: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: sequential_3/p4_conv2d_30/rot90_2/Assert/AssertGuard/branch_executed/_274


329/329 [==============================] - ETA: 0s - loss: 4.8846 - accuracy: 0.0635

2023-01-21 15:14:47.827071: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: sequential_3/p4_conv2d_27/rot90/Assert/AssertGuard/branch_executed/_8


329/329 [==============================] - 311s 926ms/step - loss: 4.8846 - accuracy: 0.0635 - val_loss: 4.3507 - val_accuracy: 0.0983
Epoch 2/50
329/329 [==============================] - 315s 958ms/step - loss: 4.2174 - accuracy: 0.1136 - val_loss: 4.1885 - val_accuracy: 0.1159
Epoch 3/50
329/329 [==============================] - 303s 921ms/step - loss: 4.0545 - accuracy: 0.1321 - val_loss: 4.1153 - val_accuracy: 0.1263
Epoch 4/50
329/329 [==============================] - 291s 887ms/step - loss: 3.9662 - accuracy: 0.1424 - val_loss: 3.9884 - val_accuracy: 0.1401
Epoch 5/50
329/329 [==============================] - 266s 810ms/step - loss: 3.8990 - accuracy: 0.1526 - val_loss: 3.9251 - val_accuracy: 0.1460
Epoch 6/50
329/329 [==============================] - 276s 840ms/step - loss: 3.8495 - accuracy: 0.1577 - val_loss: 3.8809 - val_accuracy: 0.1501
Epoch 7/50
329/329 [==============================] - 274s 833ms/step - loss: 3.8098 - accuracy: 0.1629 - val_loss: 3.8549 - val_accura

KeyboardInterrupt: 